In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import your ResNet-20 model
from resnet20_baseline import build_resnet20  # Make sure resnet20_baseline.py is in the same folder or sys.path

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True,
    fill_mode='reflect'
)
datagen.fit(x_train)

In [ ]:
# Learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[100, 150],
    values=[0.1, 0.01, 0.001]
)

# Optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Build and compile model
model = build_resnet20()
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    validation_data=(x_test, y_test),
    epochs=200,
    verbose=1
)

In [ ]:
# Save model to specified directory
save_path = "/Users/anshbharti/Documents/SEM-6TH/Research_Paper/cnn_pruning_cifar10/results/baseline_resnet20.h5"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
model.save(save_path)
print(f"Baseline model saved at: {save_path}")

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("ResNet-20 Baseline Training")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()